# INTRODUCTION

### This notebook provides a step-by-step guide to running the student facial recognition model.

* The model can be used to identify one or more faces in an image. In detail, it has a preprocessing step that can crop out one or more faces in an image. This is achieved using a series of neural networks to identify faces, define a bounding box around the faces and crop according to the bounding box .

* The cropped (processed) images are then fed to another model to classify each cropped image.

* A step by step guide is provided to train and test the image

## Link Folder

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/PhD/faith_project

Mounted at /content/drive
/content/drive/MyDrive/PhD/faith_project


## Import necessary libraries and dependencies

In [2]:
from preprocessing.src import detect_faces, show_bboxes
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

## Process images

In [3]:
def extract_and_visualise_faces_from_image(image_path, reduce_resolution=False, reduction_ratio=0.7, resample=Image.NEAREST, enlargement_percentage = 0.9):
    img_resized = Image.open(image_path)
    plt.imshow(img_resized)
    #print("img_resized:  ", img_resized.size)
    plt.show()
    new_width = int(img_resized.width*reduction_ratio)
    new_height = int(img_resized.height*reduction_ratio)
    #print(new_width, new_height)
    img_resized = img_resized.resize((new_width, new_height), resample=Image.NEAREST)
    # print("Resized.......")
    # plt.imshow(img_resized)
    # plt.show()
    #print("img_resized:  ", img_resized.size)
    bounding_boxes, landmarks = detect_faces(img_resized, min_face_size=30.0)
    #print("bounding box shape: ", bounding_boxes.shape)
    #print("done.......")
    img_resized_with_bb = show_bboxes(img_resized, bounding_boxes, landmarks)
    plt.imshow(img_resized_with_bb)
    plt.show()
    #print("done.......")
    #image_name = image_path.split('.')[0]
    image_name = image_path.replace("unprocessed_", "").split(".")[0]
    new_image_path = []
    for i, box in enumerate(bounding_boxes):
        x1, y1, x2, y2, _ = box
        width = x2 - x1
        height = y2 - y1

        # Calculate enlargement amounts
        x_enlargement = width * enlargement_percentage / 2
        y_enlargement = height * enlargement_percentage / 2

        # Adjust bounding box coordinates
        x1_enlarged = int(max(0, x1 - x_enlargement))
        y1_enlarged = int(max(0, y1 - y_enlargement))
        x2_enlarged = int(min(img_resized.width, x2 + x_enlargement))
        y2_enlarged = int(min(img_resized.height, y2 + y_enlargement))

        cropped_face = img_resized.crop((x1_enlarged, y1_enlarged, x2_enlarged, y2_enlarged))

        output_filename = f"{image_name}_0{i+1}.jpg"  # Or .
        #output_filename = output_filename.replace("datasets/", "")
        new_image_path.append(output_filename)
        print(f"new image path: {output_filename}")
        plt.imshow(cropped_face)
        plt.show()
        cropped_face.save(output_filename)
        return new_image_path



def process_text_file(unprocessed_filepath, line_processor):
  """
  Reads a text file line by line and applies a processing function to each line.

  Args:
      filepath: The path to the text file.
      line_processor: A function that takes a single line (string) as input and
                      performs some operation on it.
  """
  try:
    with open(unprocessed_filepath, 'r') as file:
      for i, line in enumerate(file):
        # Remove leading/trailing whitespace, including newline characters.

        if i >= 49 and i <= 59 :
          print("\n")
          unprocessed_image_path = line.strip()
          if i == 0:
            mode = "w"
          else:
            mode = "a"

          # if i>2:
          #   continue

          filepath = unprocessed_filepath.replace("unprocessed_", "")

          unprocessed_image_path = "datasets/" + unprocessed_image_path
          print("unprocessed_image_path:", unprocessed_image_path)
          image_paths = line_processor(unprocessed_image_path)
          print("image_paths: ", image_paths)
          for im_path in image_paths:
            im_path = im_path.replace("datasets/", "")
            #print("new_image_path: ", im_path)
          # image_path = unprocessed_image_path.replace("unprocessed_", "")


            #print(filepath)
            # Apply the line processing function to each line.
            with open(filepath, mode) as file:
              #print(image_path)
              file.write(im_path + '\n')
            #print("done.............")
            #line_processor(cleaned_line)

  except FileNotFoundError:
    print(f"Error: File not found at {filepath}")
  except Exception as e:
    print(f"An error occurred: {e}")

# TRAINING

## Load train and val data

### STEP 1:

* The dataset is divided into train, validation and test. The training data is used to train the model while the validation (val) data is used to test the model during training. The test data is used to test for inference, i.e., after training.

* Upload your train and validation images in /faith_project/datasets/VGG_Faces2/unprocessed_train and /faith_project/datasets/VGG_Faces2/unprocessed_val, respectively.

* All images of the same student or class MUST be in the respective folders.

* The default "unprocessed_train" and "unprocessed_val" folder have 10 sub-folders (sub-folder name: n000001,n000002,...,n000010), representing 10 classes (students).

* If you want to increase the number of student classes; add more folders to the train and test respectively following the folder name format.

* Ensure that all images are in their correct folder.

### STEP 2:
* Update the meta_data.csv file with the sub-folder name for each student and the name of the student.
* Each entry has the folder name (i.e., n00001), Name (John_Doe) and Flag (1).

### STEP 3:
*  Update the train_image_file_list.txt and val_image_file_list.txt files with the location of all train and val images, respectively. (**IMPORTANT**)

* Default **train** file list have 100 lines of file location, representing 10 images for each of the 10 students.

* Default **val** file list have 40 lines of file location, representing 4 images for each of the 10 students.

## Preprocess train and val datasets

In [ ]:
filepath = "unprocessed_train_image_file_list.txt"
process_text_file(filepath, extract_and_visualise_faces_from_image)

In [ ]:
 filepath = "unprocessed_val_image_file_list.txt"
 process_text_file(filepath, extract_and_visualise_faces_from_image)

## Train Model

In [ ]:
%run demo.py train --arch_type resnet50_ft --num_classes 10 --train_img_list_file "train_image_file_list.txt" --test_img_list_file "val_image_file_list.txt" --meta_file "meta_data.csv" --dataset_dir "datasets" --feature_dir "datasets" --log_file "log_train.txt" --config 1 --batch_size 10  --checkpoint_dir "checkpoint_1" --resume "checkpoint/checkpoint.pth.tar"

Namespace(cmd='train', arch_type='resnet50_ft', num_classes=10, dataset_dir='datasets', log_file='log_train.txt', train_img_list_file='train_image_file_list.txt', test_img_list_file='val_image_file_list.txt', meta_file='meta_data.csv', checkpoint_dir='checkpoint_1', feature_dir='datasets', config=1, batch_size=10, resume='checkpoint/checkpoint.pth.tar', weight_file='models/resnet50_ft_weight.pkl', gpu=0, workers=4, horizontal_flip=False)
df:    Class_ID      Name  Flag
0  n000001  Person B     1
1  n000002  Person C     1
2  n000003  Person D     1
3  n000004  Person E     1
4  n000005  Person F     1
5  n000006  Person G     1
6  n000007  Person H     1
7  n000008  Person J     1
8  n000009  Person K     1
9  n000010  Person L     1
id_label_dict:  {'n000001': np.int64(0), 'n000002': np.int64(1), 'n000003': np.int64(2), 'n000004': np.int64(3), 'n000005': np.int64(4), 'n000006': np.int64(5), 'n000007': np.int64(6), 'n000008': np.int64(7), 'n000009': np.int64(8), 'n000010': np.int64(9)}

Train:   0%|                                              | 0/3 [00:00<?, ?it/s]

epoch:  3
self.iteration:  51
self.max_iter:  70



Train:  33%|████████████▋                         | 1/3 [00:04<00:09,  4.59s/it]

Train_summary: [12/13/0]	epoch: 3	iter: 51	Time: 0.000	Data: 2.422	Loss: 0.0000	Prec@1: 0.000	Prec@5: 0.000	lr 0.000250
epoch:  4
self.iteration:  51
self.max_iter:  70


...

output:  tensor([6, 6, 7, 4, 7, 7, 4, 7, 7, 8])
target:  tensor([6, 6, 7, 7, 7, 7, 7, 7, 7, 8])
Test: [5/7/60]	epoch: 5	iter: 77	Time: 2.207 (2.454)	Loss: 0.3816 (0.0967)	Prec@1: 80.000 (95.000)	Prec@5: 100.000 (100.000)	




Valid iteration=77 epoch=5: 100%|█████████████████| 7/7 [00:16<00:00,  2.13s/it]

                                                                                

output:  tensor([8, 8, 8, 9, 9, 9, 9])
target:  tensor([8, 8, 8, 9, 9, 9, 9])
Test: [6/7/67]	epoch: 5	iter: 77	Time: 1.624 (2.335)	Loss: 0.0101 (0.0877)	Prec@1: 100.000 (95.522)	Prec@5: 100.000 (100.000)	
Test_summary: [6/7/67] epoch: 5 iter: 77	BestPrec@1: 95.522	BestPrec@5: 100.000	Time: 2.335	Loss: 0.0877	Prec@1: 95.522	Prec@5: 100.000	



Train:  67%|█████████████████████████▎            | 2/3 [01:51<00:55, 55.91s/it]

Train: [12/13/122]	epoch: 5	iter: 77	Time: 17.568 (4.034)	Data: 0.035 (0.113)	Loss: 0.2015 (0.0045)	Prec@1: 100.000 (100.000)	Prec@5: 100.000 (100.000)	lr 0.000125
Train_summary: [12/13/122]	epoch: 5	iter: 77	Time: 4.034	Data: 0.113	Loss: 0.0045	Prec@1: 100.000	Prec@5: 100.000	lr 0.000125


# TEST


## Load test data

### STEP 1:
* Upload your test images in /faith_project/datasets/VGG_Faces2/unprocessed_test.
* Default "unprocessed_test" folder has only 1 subfloder with all images.

### STEP 2:

* Update the meta_data.csv file with the sub-folder name for each student and the name of the student.

* Each entry has the folder name (i.e., n00001), Name (John_Doe) and Flag (1).

### STEP 3:
*  Update the test_image_file_list.txt  files with the location of all test iamges. (**IMPORTANT**)

* Default **test** file list have 10 lines of file location, representing 1 images for each of the 10 students.


## Preprocess test images

In [ ]:
filepath = "unprocessed_test_image_file_list.txt"
process_text_file(filepath, extract_and_visualise_faces_from_image)

## Test model

In [5]:
%run demo.py test --arch_type resnet50_ft --num_classes 10 --test_img_list_file "test_image_file_list.txt" --meta_file "meta_data.csv" --dataset_dir "datasets" --feature_dir "datasets" --log_file "log_test.txt" --resume "checkpoint/model_best.pth.tar"

Namespace(cmd='test', arch_type='resnet50_ft', num_classes=10, dataset_dir='datasets', log_file='log_test.txt', train_img_list_file='/path/to/train_image_list.txt', test_img_list_file='test_image_file_list.txt', meta_file='meta_data.csv', checkpoint_dir='/path/to/checkpoint_directory', feature_dir='datasets', config=1, batch_size=32, resume='checkpoint/model_best.pth.tar', weight_file='/path/to/weight_file.pkl', gpu=0, workers=4, horizontal_flip=False)
df:    Class_ID      Name  Flag
0  n000001  Person B     1
1  n000002  Person C     1
2  n000003  Person D     1
3  n000004  Person E     1
4  n000005  Person F     1
5  n000006  Person G     1
6  n000007  Person H     1
7  n000008  Person J     1
8  n000009  Person K     1
9  n000010  Person L     1
id_label_dict:  {'n000001': np.int64(0), 'n000002': np.int64(1), 'n000003': np.int64(2), 'n000004': np.int64(3), 'n000005': np.int64(4), 'n000006': np.int64(5), 'n000007': np.int64(6), 'n000008': np.int64(7), 'n000009': np.int64(8), 'n000010

output:  tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
target:  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Test: [0/1/10]	epoch: 0	iter: 0	Time: 4.589 (4.589)	Loss: 6.8810 (6.8810)	Prec@1: 10.000 (10.000)	Prec@5: 70.000 (70.000)	
